In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import statsmodels.api as sm
import itertools

# plt.style.use('fivethirtyeight')
# %matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 13, 8

import plotly.express as px

# import collections

import functions as fun


import datetime
import seaborn as sns
sns.set()


In [2]:
%load_ext autoreload
%autoreload

In [3]:
# plotting dfs:
('roi_df_plotting.csv')
('top_percent_roi_plotting.csv')
('chi_plot.csv')

# modeling dfs:
('top_roi_for_melting.csv')
('chi_roi_for_melting.csv')
('top_zips_with_all_columns.csv')

'top_zips_with_all_columns.csv'

In [4]:
df = pd.read_csv('csv/top_zips_with_all_columns.csv')
df.head()

,RegionID,ZipCode,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
2,84640,60640,Chicago,IL,Chicago,Cook,8,216500.0,216700.0,216900.0,...,798000,787100,776100,774900,777900,777900,778500,780500,782800,782800
3,84646,60647,Chicago,IL,Chicago,Cook,17,122700.0,122800.0,122800.0,...,459600,461400,464300,466500,467900,470600,474500,475100,472600,470200
4,84620,60618,Chicago,IL,Chicago,Cook,20,142600.0,143100.0,143400.0,...,462700,461200,459900,459200,458700,457900,457400,459000,462500,464300


In [5]:
zips_df = pd.read_csv('csv/uszips.csv')
zips_df.tail()

,zip,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,county_fips,county_name,county_weights,county_names_all,county_fips_all,imprecise,military,timezone
33092,99923,55.97796,-130.03671,Hyder,AK,Alaska,True,NaN,15,2.1,2198,Prince of Wales-Hyder,{'02198':100},Prince of Wales-Hyder,02198,False,False,America/Sitka
33093,99925,55.55767,-132.97627,Klawock,AK,Alaska,True,NaN,927,5.7,2198,Prince of Wales-Hyder,{'02198':100},Prince of Wales-Hyder,02198,False,False,America/Sitka
33094,99926,55.12617,-131.48928,Metlakatla,AK,Alaska,True,NaN,1635,4.2,2198,Prince of Wales-Hyder,{'02198':100},Prince of Wales-Hyder,02198,False,False,America/Metlakatla
33095,99927,56.25100,-133.37571,Point Baker,AK,Alaska,True,NaN,38,0.2,2198,Prince of Wales-Hyder,{'02198':100},Prince of Wales-Hyder,02198,False,False,America/Sitka
33096,99929,56.36950,-131.93648,Wrangell,AK,Alaska,True,NaN,2484,0.4,2275,Wrangell,{'02275':100},Wrangell,02275,False,False,America/Sitka


In [6]:
# making a dictionary of zip codes with their corresponding region id's 
# because I may need them for plotting or identification of zip code location
# region
region_dict = {}
zip_list = df.ZipCode.unique()
for i in zip_list:
    region = df[df['ZipCode']==i].RegionID.values[0]
    region_dict[i] = region

In [7]:
region_dict[60647]

84646

In [8]:
zip_list

array([60657, 60614, 60640, 60647, 60618, 60613, 60629, 60625, 60619,
       60610, 60620, 60634, 60626, 60649, 60616, 60622, 60615, 60641,
       60660, 60608])

In [9]:
neighborhood_list = []

In [10]:
zipcode_df = []
for z in zip_list:
    zipcode_df.append(zips_df.loc[zips_df['zip']==z])

zipcode_df = pd.concat(zipcode_df)

In [11]:
zipcode_df.head()

,zip,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,county_fips,county_name,county_weights,county_names_all,county_fips_all,imprecise,military,timezone
20732,60657,41.93998,-87.65374,Chicago,IL,Illinois,True,NaN,70052,11207.9,17031,Cook,{'17031':100},Cook,17031,False,False,America/Chicago
20693,60614,41.92280,-87.65139,Chicago,IL,Illinois,True,NaN,71308,8244.5,17031,Cook,{'17031':100},Cook,17031,False,False,America/Chicago
20717,60640,41.97236,-87.66347,Chicago,IL,Illinois,True,NaN,69715,10530.6,17031,Cook,{'17031':100},Cook,17031,False,False,America/Chicago
20724,60647,41.92068,-87.70167,Chicago,IL,Illinois,True,NaN,87509,8385.0,17031,Cook,{'17031':100},Cook,17031,False,False,America/Chicago
20697,60618,41.94696,-87.70262,Chicago,IL,Illinois,True,NaN,94395,7114.6,17031,Cook,{'17031':100},Cook,17031,False,False,America/Chicago


In [12]:
zipcode_df.to_csv('zipcodes_info.csv', index=False)

In [13]:
zips_df.loc[zips_df['zip']==60647]

,zip,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,county_fips,county_name,county_weights,county_names_all,county_fips_all,imprecise,military,timezone
20724,60647,41.92068,-87.70167,Chicago,IL,Illinois,True,NaN,87509,8385.0,17031,Cook,{'17031':100},Cook,17031,False,False,America/Chicago


In [14]:
# region_dict[60657]

In [15]:
# get locations
# best_zips = list(top_zips_df.ZipCode.values)
for i in zip_list:
    city = zipcode_df[zipcode_df['zip']==i].city.values[0]
    state = zipcode_df[zipcode_df['zip']==i].state_id.values[0]
    county_fips = zipcode_df[zipcode_df['zip']==i].county_fips.values[0]
    density = zipcode_df[zipcode_df['zip']==i].density.values[0]
    print(f'ZipCode: {i}\n fips: {county_fips}')
    print(f'density: {density}\n')

ZipCode: 60657
 fips: 17031
density: 11207.9

ZipCode: 60614
 fips: 17031
density: 8244.5

ZipCode: 60640
 fips: 17031
density: 10530.6

ZipCode: 60647
 fips: 17031
density: 8385.0

ZipCode: 60618
 fips: 17031
density: 7114.6

ZipCode: 60613
 fips: 17031
density: 8572.6

ZipCode: 60629
 fips: 17031
density: 6466.2

ZipCode: 60625
 fips: 17031
density: 7837.6

ZipCode: 60619
 fips: 17031
density: 4092.5

ZipCode: 60610
 fips: 17031
density: 12585.9

ZipCode: 60620
 fips: 17031
density: 3935.6

ZipCode: 60634
 fips: 17031
density: 4033.0

ZipCode: 60626
 fips: 17031
density: 11355.1

ZipCode: 60649
 fips: 17031
density: 6148.9

ZipCode: 60616
 fips: 17031
density: 4677.4

ZipCode: 60622
 fips: 17031
density: 8213.0

ZipCode: 60615
 fips: 17031
density: 7086.4

ZipCode: 60641
 fips: 17031
density: 6845.9

ZipCode: 60660
 fips: 17031
density: 12796.8

ZipCode: 60608
 fips: 17031
density: 5068.9



In [16]:
# fig = px.choropleth(zips_df, lat='lat', lon='lng', locationmode='USA-states',
#                      scope='usa', color_continuous_scale='Purples', 
#             title='Percent Yearly Average ROI by State')
# fig.show()

In [17]:
# fig = px.choropleth(
#   fips = county_fips, values = denty, colorscale = heatmap_color,
#   show_state_data = True, scope = ['IL'],
#   binning_endpoints=endpts,
#   county_outline={'color': 'rgb(15,15,55)', 'width': 1},
#   state_outline={'color': 'rgb(15,15,55)', 'width': 1},
#   legend_title='Median Rent', title='Median Rent per Square Foot by County'
#  )

In [19]:
# zip_df.info()

In [21]:
# import plotly.figure_factory as ff

# # fips = ['06021', '06023', '06027',
# #         '06029', '06033', '06059',
# #         '06047', '06049', '06051',
# #         '06055', '06061']
# # values = range(len(fips))

# # fig = ff.create_choropleth(fips=zips_df.county_fips, values=range(len(zips_df.county_fips))
# fig = ff.create_choropleth(fips=zips_df.county_fips, values=range(len(zips_df.county_fips))
# fig.layout.template = None
# fig.show()                           
                          
